In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [22]:
def request_page(page_number):
    url = f'https://blog.mozilla.org/page/{page_number}/'
    response = requests.get(url)
    html= response.text
    return BeautifulSoup(html, 'html.parser')

def parse_wordpress(soup):
    articles = soup.find_all('article', {'class': 'post-summary'})
    for article in articles:
        yield {
            'title': article.find('h2', {'class': 'entry-title'}).text.strip(),
            'wordpress url': article.find('a', {'class': 'entry-link'})['href'],
            'author': article.find('a', {'class': 'author url fn'}).text.strip(),
            'author url': article.find('a', {'class': 'author url fn'})['href'],
            'date published': datetime.fromisoformat(
                article.find('time', {'class': 'date published'})['datetime']
            ),
            
        }
        
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [23]:
soup = request_page(1)

In [24]:
wordpress = parse_wordpress(soup)

In [25]:
save_to_csv(wordpress, 'page1_wordpress.csv')

In [26]:
import itertools

In [27]:
wordpress_data = itertools.chain.from_iterable(parse_wordpress(request_page(page_number)) 
                              for page_number in range(84, 87))
save_to_csv(wordpress_data, 'wordpress.csv')